In [1]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

x_data = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y_data = np.array([[0], [1], [1], [0]], dtype=np.float32)

X = tf.placeholder(tf.float32, [None, 2], name='x-input')
Y = tf.placeholder(tf.float32, [None, 1], name='y-input')
# Use Neural Network

# layer1
with tf.name_scope("layer1") as scope:
    W1 = tf.Variable(tf.random_normal([2,2]),name="weight1")
    b1 = tf.Variable(tf.random_normal([2]),name="bias1")
    layer1 = tf.sigmoid(tf.matmul(X,W1)+b1)

    w1_hist = tf.summary.histogram("weights1", W1)
    b1_hist = tf.summary.histogram("biases1", b1)
    layer1_hist = tf.summary.histogram("layer1", layer1)
    
# layer2
with tf.name_scope("layer2") as scope:
    W2 = tf.Variable(tf.random_normal([2,2]),name="weight2")
    b2 = tf.Variable(tf.random_normal([2]),name="bias2")
    layer2 = tf.sigmoid(tf.matmul(layer1,W2)+ b2)
    
    w2_hist = tf.summary.histogram("weights2", W2)
    b2_hist = tf.summary.histogram("biases2", b2)
    layer2_hist = tf.summary.histogram("layer2", layer2)

# layer3
with tf.name_scope("layer3") as scope:
    W3 = tf.Variable(tf.random_normal([2,1]),name="weight3")
    b3 = tf.Variable(tf.random_normal([1]),name="bias3")
    hypothesis = tf.sigmoid(tf.matmul(layer2,W3)+ b3)
    
    w3_hist = tf.summary.histogram("weights3", W3)
    b3_hist = tf.summary.histogram("biases3", b3)
    hypothesis_hist = tf.summary.histogram("hypothesis", hypothesis)


with tf.name_scope("cost") as scope:
    cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
    cost_summ = tf.summary.scalar("cost", cost)

with tf.name_scope("train") as scope:
    train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.cast(hypothesis > 0.5,dtype=tf.float32)
is_correct = tf.equal(prediction,Y)
accuracy = tf.reduce_mean(tf.cast(is_correct,dtype=tf.float32))
accuracy_summ = tf.summary.scalar("accuracy", accuracy)


with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_logs
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/NN1_log")
    writer.add_graph(sess.graph) # Show the graph
    
    sess.run(tf.global_variables_initializer())
    
    for step in range(10001):
        summary, _ = sess.run([merged_summary,train],feed_dict={
            X : x_data,
            Y : y_data
        })
        writer.add_summary(summary, global_step=step)
            
    h,p,acc = sess.run([hypothesis,prediction,accuracy],feed_dict={
        X : x_data,
        Y : y_data
    })
    print("hypo : ",h)
    print("Predict : ",p)
    print("Accuracy : ",acc)


hypo :  [[ 0.01055974]
 [ 0.99349725]
 [ 0.99051219]
 [ 0.00899102]]
Predict :  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]
Accuracy :  1.0
